In [1]:
import pandas as pd
import keras
import keras_tuner

from textblob import TextBlob
from keras_tuner import RandomSearch
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler

2024-06-12 20:23:49.324814: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 20:23:49.324880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 20:23:49.325824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-12 20:23:49.333433: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-12 20:23:50.295946: W tensorflow/compiler/tf2

In [2]:
def get_sentiment(text):
    if text:
        return TextBlob(text).sentiment.polarity
    else:
        return 0

In [3]:
data = pd.read_csv('combined-dataset/places-dataset.csv')
data

,id,name,latitude,longitude,address,url,status,phone,primary-type,types,rating,rating-count,price-level,review 1,review 2,review 3,review 4,review 5
0,ChIJYcGr7GSb0S0RckePBrCWikw,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,-8.103137,115.090514,"Jl. Surapati No.12 14, Kp. Baru, Kec. Buleleng...",https://maps.google.com/?cid=5515386376437057394,OPERATIONAL,+62 362 3301500,hotel,"hotel, lodging",4.1,2581.0,NaN,"It has quite small room, and the hallway is qu...",Few days ago My wife and me stayed 2 night at ...,"Because its cheap, cant complaint too much. Th...","Stayed here for 1 night, the room is small wit...",Good place to short stay located next Historic...
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,Warung Bik Juk,-8.106189,115.089961,"Kampung Kajanan, Buleleng, Buleleng Regency, B...",https://maps.google.com/?cid=2294119279772554194,OPERATIONAL,+62 819-1566-2913,indonesian_restaurant,"indonesian_restaurant, restaurant, food",4.5,648.0,PRICE_LEVEL_MODERATE,"Surprisingly, a really good warung that’s hidd...","So in love with the food, very simple but tast...",A hidden treasure in the heart of Singaraja Ci...,Nice warung on the side street. They were out ...,Nostalgic place at singaraja.. good and friend...
2,ChIJYyHbhgia0S0RzdjNXLmcf54,Harbour Tourist Area of Buleleng,-8.103931,115.090031,"Jl. Pelabuhan Buleleng, Kp. Bugis, Kec. Bulele...",https://maps.google.com/?cid=11421019499974744269,OPERATIONAL,+62 362 21342,tourist_attraction,"tourist_attraction, restaurant, food",4.3,2800.0,NaN,"Only had a fleeting visit here, came by coach,...",The Harbour Tourist Area in Buleleng offers a ...,Its ine of nice place you must visit when you ...,"A historical place for the Balinese people, es...",It's a good place to relax with you friends an...
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,Abuela Coffee,-8.104751,115.087919,"Jl. Erlangga No.1, Kp. Bugis, Kec. Buleleng, K...",https://maps.google.com/?cid=10868728212674968854,OPERATIONAL,+62 858-5878-5361,coffee_shop,"coffee_shop, cafe, food, store",4.9,164.0,NaN,"One word, underrated! How come place like this...",Coffee shop with good food and coffee. Very ni...,"Small cafe, the coffee is good, the dimsum is ...",Love this place 🫶\r\nGreat coffee ☕️\r\nLumpia...,Best coffee in Singaraja. Cappuccino was 20k a...
4,ChIJxaITmQia0S0RyrbukE8vsJU,Klenteng Ling Gwan Kiong,-8.103899,115.089833,"Jl. Erlangga No.65, Kp. Bugis, Kec. Buleleng, ...",https://maps.google.com/?cid=10786173126332823242,OPERATIONAL,NaN,place_of_worship,"tourist_attraction, place_of_worship",4.6,142.0,NaN,"This temple is located in Singaraja, located i...",Taoist temple with beautiful old-chinese archi...,Ling Gwan Kiong is an old Chinese temple popul...,"The Tri Dharma Temple, very unique, historic a...","Quiet, clean and peaceful. Visited this place ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6972,ChIJf6d0f5Nb0i0Rq_tEZeZB_Nc,Melasti Beach Ungasan,-8.847742,115.160670,"Indonesia, Bali, Badung Regency, South Kuta, U...",https://maps.google.com/?cid=15563386870082894763,OPERATIONAL,NaN,tourist_attraction,tourist_attraction,4.8,19752.0,NaN,the location is not far and not too crowded. t...,Nicely made beach with parking right in front ...,Melasti Beach Ungasan is a stunning beach in B...,The approach road to this beach is amazing spe...,"🚶 I walked down to the beach, but it’s quite s..."
6973,ChIJk41ld9Fb0i0RStb1NFpDAno,Tropical Temptation Beach Club by Ini Vie Hosp...,-8.848187,115.160380,"Melasti Beach 88, Ungasan, Kec. Kuta Sel., Kab...",https://maps.google.com/?cid=8791663477295273546,OPERATIONAL,+62 812-3776-3903,restaurant,"restaurant, bar, food",4.6,2990.0,NaN,"First off, the staff is incredible. Like, rid...",We ended up paying 1.5mil for two here while s...,Had an awesome day at Tropical Temptation! Th...,"Beautiful area, can not fault how breath takin...",Tropical Temptation is a beach club gem! 💎 The...
6974,ChIJO30FJJdd0i0RCOyG8WcyJBk,Pantai Melasti,-8.848278,115.163173,"554Q+PJ5, Jl. Pantai Melasti,

In [4]:
data.dropna(subset=['primary-type'], inplace=True)
data.dropna(subset=['review 1', 'review 2', 'review 3', 'review 4', 'review 5'], inplace=True)
data.fillna(value={'price-level': 'PRICE_LEVEL_UNKNWON'}, inplace=True)
print(data.isnull().sum())

id                0
name              0
latitude          0
longitude         0
address           0
url               0
status            0
phone           811
primary-type      0
types             0
rating            0
rating-count      0
price-level       0
review 1          0
review 2          0
review 3          0
review 4          0
review 5          0
dtype: int64


In [5]:
data['sentiment_review1'] = data['review 1'].apply(get_sentiment)
data['sentiment_review2'] = data['review 2'].apply(get_sentiment)
data['sentiment_review3'] = data['review 3'].apply(get_sentiment)
data['sentiment_review4'] = data['review 4'].apply(get_sentiment)
data['sentiment_review5'] = data['review 5'].apply(get_sentiment)
data['sentiment_score'] = data[['sentiment_review1', 'sentiment_review2', 'sentiment_review3', 'sentiment_review4', 'sentiment_review5']].mean(axis=1)

In [6]:
scaler = MinMaxScaler()
data['normalized_sentiment'] = scaler.fit_transform(data[['sentiment_score']])
data['normalized_rating'] = scaler.fit_transform(data[['rating']])
data['normalized_rating_count'] = scaler.fit_transform(data[['rating-count']])

In [7]:
data

,id,name,latitude,longitude,address,url,status,phone,primary-type,types,...,review 5,sentiment_review1,sentiment_review2,sentiment_review3,sentiment_review4,sentiment_review5,sentiment_score,normalized_sentiment,normalized_rating,normalized_rating_count
0,ChIJYcGr7GSb0S0RckePBrCWikw,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,-8.103137,115.090514,"Jl. Surapati No.12 14, Kp. Baru, Kec. Buleleng...",https://maps.google.com/?cid=5515386376437057394,OPERATIONAL,+62 362 3301500,hotel,"hotel, lodging",...,Good place to short stay located next Historic...,0.012500,0.322240,0.251905,0.127778,2.220446e-17,0.142885,0.373328,0.400000,0.041096
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,Warung Bik Juk,-8.106189,115.089961,"Kampung Kajanan, Buleleng, Buleleng Regency, B...",https://maps.google.com/?cid=2294119279772554194,OPERATIONAL,+62 819-1566-2913,indonesian_restaurant,"indonesian_restaurant, restaurant, food",...,Nostalgic place at singaraja.. good and friend...,0.212500,0.141667,0.308333,0.132986,2.078571e-01,0.200669,0.426269,0.666667,0.009710
2,ChIJYyHbhgia0S0RzdjNXLmcf54,Harbour Tourist Area of Buleleng,-8.103931,115.090031,"Jl. Pelabuhan Buleleng, Kp. Bugis, Kec. Bulele...",https://maps.google.com/?cid=11421019499974744269,OPERATIONAL,+62 362 21342,tourist_attraction,"tourist_attraction, restaurant, food",...,It's a good place to relax with you friends an...,-0.016667,0.284145,0.561538,0.059091,7.191358e-02,0.192004,0.418331,0.533333,0.044652
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,Abuela Coffee,-8.104751,115.087919,"Jl. Erlangga No.1, Kp. Bugis, Kec. Buleleng, K...",https://maps.google.com/?cid=10868728212674968854,OPERATIONAL,+62 858-5878-5361,coffee_shop,"coffee_shop, cafe, food, store",...,Best coffee in Singaraja. Cappuccino was 20k a...,0.344676,0.740000,0.204167,0.457813,5.800000e-01,0.465331,0.668747,0.933333,0.001851
4,ChIJxaITmQia0S0RyrbukE8vsJU,Klenteng Ling Gwan Kiong,-8.103899,115.089833,"Jl. Erlangga No.65, Kp. Bugis, Kec. Buleleng, ...",https://maps.google.com/?cid=10786173126332823242,OPERATIONAL,NaN,place_of_worship,"tourist_attraction, place_of_worship",...,"Quiet, clean and peaceful. Visited this place ...",0.139524,0.603095,0.016667,0.167500,2.458333e-01,0.234524,0.457286,0.733333,0.001494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6972,ChIJf6d0f5Nb0i0Rq_tEZeZB_Nc,Melasti Beach Ungasan,-8.847742,115.160670,"Indonesia, Bali, Badung Regency, South Kuta, U...",https://maps.google.com/?cid=15563386870082894763,OPERATIONAL,NaN,tourist_attraction,tourist_attraction,...,"🚶 I walked down to the beach, but it’s quite s...",0.150556,0.376905,0.234114,0.317460,1.644006e-01,0.248687,0.470262,0.866667,0.319905
6973,ChIJk41ld9Fb0i0RStb1NFpDAno,Tropical Temptation Beach Club by Ini Vie Hosp...,-8.848187,115.160380,"Melasti Beach 88, Ungasan, Kec. Kuta Sel., Kab...",https://maps.google.com/?cid=8791663477295273546,OPERATIONAL,+62 812-3776-3903,restaurant,"restaurant, bar, food",...,Tropical Temptation is a beach club gem! 💎 The...,0.450000,0.184188,0.519444,0.006250,6.178571e-01,0.355548,0.568166,0.733333,0.047737
6974,ChIJO30FJJdd0i0RCOyG8WcyJBk,Pantai Melasti,-8.848278,115.163173,"554Q+PJ5, Jl. Pantai Melasti, Kutuh, Kec. Kuta...",https://maps.google.com/?cid=1811628372124953608,OPERATIONAL,NaN,tourist_attraction,tourist_attraction,...,it was really hot when we went here but its a ...,0.101190,0.633333,0.287500,0.850000,2.833333e-01,0.431071,0.637359,0.800000,0.000406
6975,ChIJvazdGXtb0i0RA-4XopPth7A,Mïnoo Beach Club,-8.848279,115.161177,"Jl. Pengayutan. Pantai Melasti, Ungasan, Kec. ...",https://maps.google.com/?cid=12720396890866707971,OPERATIONAL,+62 811-3898-902,bar,"bar, restaurant, food",...,"I went to this beach a few days ago, the food ...",0.483333,0.441667,0.438333,0.355000,6.466875e-01,0.473004,0.675777,0.666667,0.010960


In [8]:
mlb = MultiLabelBinarizer()
types_matrix = mlb.fit_transform(data['types'].str.split(', '))
types_df = pd.DataFrame(types_matrix, columns=mlb.classes_, index=data.index)

In [9]:
features = pd.concat([
    types_df,
    data[['normalized_rating', 'normalized_rating_count', 'normalized_sentiment']]
], axis=1)

In [10]:
labels = (data['rating'].astype(int))

In [11]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=42)

In [12]:
# input_shape = features.shape[1]
# 
# model = models.Sequential([
#     layers.Input(shape=(input_shape,)),
#     layers.Dense(128, activation='relu'),
#     layers.Dropout(0.4),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])
# 
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

In [13]:
# model.fit(train_features, train_labels, epochs=25, validation_data=(val_features, val_labels))

In [14]:
# model.predict()

In [15]:
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Input(shape=(features.shape[1],)))
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=1024, step=32), activation='relu'))
        model.add(layers.Dropout(hp.Float('dropout_' + str(i), 0, 0.5, step=0.1)))
    model.add(layers.Dense(25, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='model-testing',
    project_name='places-rating'
)

Reloading Tuner from model-testing/places-rating/tuner0.json


In [17]:
tuner.search(train_features, train_labels, epochs=5, validation_data=(val_features, val_labels))


Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |1                 |num_layers
448               |64                |units_0
0                 |0.3               |dropout_0
960               |736               |units_1
0.2               |0.4               |dropout_1



2024-06-12 20:24:00.320171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 20:24:00.350138: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 20:24:00.350221: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 20:24:00.354151: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-12 20:24:00.354512: I external/local_xla/xla/stream_executor

Epoch 1/5


Traceback (most recent call last):
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, *

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/martinus/anaconda3/envs/capstone/lib/python3.11/site-packages/keras_tuner/src/engine/tuner.py", line 233, in _bui

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hyperparameters.values)
print("Best Model Summary:")
best_model.summary()


best_model.predict(features.iloc[[0]])

In [ ]:
features.iloc[0]